In [8]:
from collections import defaultdict, Counter
from IPython.display import display, clear_output
from queue import Queue
from statistics import median, mean
from time import sleep
from typing import Union
import chess
import json
import random

In [9]:
def is_move_losing(board: chess.Board, move: chess.Move) -> bool:
    """It check's if after the move the opponent has any move (it's stalmate) or if the opponent can capture queen"""
    board.push(move)
    return not board.legal_moves or any(
        True
        for m in board.legal_moves
        if m is not None and m.to_square == move.to_square
    )


def is_move_winning(board: chess.Board, move: chess.Move) -> bool:
    """Simple checking if the move is winning the game"""
    board.push(move)
    try:
        outcome: bool = board.outcome().winner
    except Exception:
        outcome: bool = False
    return outcome


def is_white_king_near_edge(board: chess.Board) -> bool:
    """As the name says it's checking if the black king is on the edge of the board"""
    return any(
        edge_names in chess.square_name(board.king(chess.WHITE))
        for edge_names in ["a", "b", "g", "h", "1", "2", "7", "8"]
    )


def calculate_black_king_distance(board: chess.Board) -> float:
    """It calculates black king distance from center of the board
    We are using Euclidean distance becuase in our situation it's better than Chebyshev distance
    """
    row: int = board.king(chess.BLACK) // 8
    column: int = board.king(chess.BLACK) % 8
    return ((3.5 - row) ** 2 + (3.5 - column) ** 2) ** 0.5


def check_queen_move_quality(board: chess.Board, move: chess.Move) -> int:
    """Simple function that returns quality of the move for the queen: 1 - good, 0 - neutral, -1 - bad"""
    start_distance: float = calculate_black_king_distance(board)
    quality_of_move: int = 1
    board.push(move)
    for move in list(board.legal_moves):
        board.push(move)
        if calculate_black_king_distance(board) < start_distance:
            quality_of_move: int = -1
            board.pop()
            break
        if calculate_black_king_distance(board) == start_distance:
            quality_of_move: int = 0
        board.pop()
    return quality_of_move


def find_best_king_move(
    board: chess.Board, target_square: str, banned_squares: set = set()
) -> Union[chess.Move, None]:
    """https://en.wikipedia.org/wiki/Breadth-first_search"""

    def move_king_to_square(board: chess.Board, target: int):
        """Force move king to the target square"""
        board.set_piece_at(board.king(chess.WHITE), None)
        board.set_piece_at(target, chess.Piece(chess.KING, chess.WHITE))

    explored: set = set()
    start_square: int = board.king(chess.WHITE)
    target_square: int = chess.parse_square(target_square)
    parents: dict = {}
    q: Queue = Queue()
    q.put(start_square)
    explored.add(start_square)
    while not q.empty():
        current_square: int = q.get()
        move_king_to_square(board, current_square)
        for move in list(board.legal_moves):
            if board.piece_at(move.from_square).symbol() == "Q" or is_move_losing(
                board.copy(), move
            ):
                continue
            if move.to_square not in explored and move.to_square not in banned_squares:
                explored.add(move.to_square)
                parents[move.to_square] = current_square
                q.put(move.to_square)
                if move.to_square == target_square:
                    break
    while True:
        if target_square not in parents:
            return None
        if parents[target_square] == start_square:
            return chess.Move(start_square, target_square)
        target_square: int = parents[target_square]


def black_king_only_one_square(board: chess.Board) -> bool:
    """Checks if black king has only one legal move"""
    if board.turn == chess.WHITE:
        board.push_uci("0000")
    return board.legal_moves.count() == 1


def is_black_king_in_corner(board: chess.Board) -> bool:
    """Checks if black king is in the corner of the board"""
    return str(chess.square_name(board.king(chess.BLACK))) in [
        "a1",
        "b1",
        "a2",
        "a8",
        "a7",
        "b8",
        "h1",
        "h2",
        "g1",
        "h8",
        "h7",
        "g8",
    ]


def get_king_final_position(board: chess.Board) -> Union[str, None]:
    """Returns the best position for the white king"""
    king_square: str = chess.square_name(board.king(chess.BLACK))
    if king_square in ["a1", "b1", "a2"]:
        return "c3"
    if king_square in ["a8", "b8", "a7"]:
        return "c6"
    if king_square in ["h1", "g1", "h2"]:
        return "f3"
    if king_square in ["h8", "g8", "h7"]:
        return "f6"
    return None


def calculate_best_move(board: chess.Board) -> chess.Move:
    """Main function that calculates the best move for the white player"""
    candidate_moves: defaultdict = defaultdict(lambda: [])
    best_king_move: chess.Move = None
    if is_black_king_in_corner(board.copy()) and black_king_only_one_square(
        board.copy()
    ):
        best_king_move = find_best_king_move(
            board.copy(), get_king_final_position(board.copy())
        )
    for move in board.legal_moves:
        if is_move_winning(board.copy(), move):
            return move
        if is_move_losing(board.copy(), move):
            continue
        if move == best_king_move:
            return move
        if board.piece_at(move.from_square).symbol() == "Q":
            candidate_moves[check_queen_move_quality(board.copy(), move)].append(move)
    return random.choice(
        sorted(candidate_moves.items(), key=lambda x: x[0], reverse=True)[0][1]
    )


def make_best_black_move(board: chess.Board) -> chess.Move:
    """Simple function that moves black king to the closest square to the center"""
    output: dict = {}
    for move in board.legal_moves:
        board.push(move)
        output[calculate_black_king_distance(board)]: chess.Move = move
        board.pop()
    return output[min(output.keys())]


def show_board(board) -> None:
    """Clear output and display the board"""
    clear_output()
    sleep(0.001)
    display(board)


def playing_loop(board, auto_play=True, show_autoplay=False) -> None:
    """Game loop"""
    while board.is_game_over() == False:
        if len(board.move_stack) > 100:
            print("Draw by 50-move rule")
            break
        if board.turn == chess.WHITE:
            board.push(calculate_best_move(board))
        else:
            if auto_play:
                if show_autoplay:
                    show_board(board)
                    sleep(0.15)
                board.push(make_best_black_move(board))
                if show_autoplay:
                    show_board(board)
                    sleep(0.15)
            else:
                try:
                    show_board(board)
                    display(calculate_black_king_distance(board))
                    sleep(1)
                    player_input: str = input("Your move: ")
                    if player_input == "exit":
                        break
                    board.push_san(player_input)
                except Exception:
                    pass
    if not auto_play:
        show_board(board)


def autplay(how_many_games: int = 100, show_games_played: bool = False) -> dict:
    """Function to calculate the winrate of the white player"""
    output: dict = {}
    for i in range(how_many_games):
        board = chess.Board(fen="8/8/8/8/8/8/8/8 w - - 0 1")
        r: int = random.randint(0, 63)
        board.set_piece_at(r, chess.Piece(chess.QUEEN, chess.WHITE))
        while True:
            r: int = random.randint(0, 63)
            if board.piece_at(r) is None:
                board.set_piece_at(r, chess.Piece(chess.KING, chess.WHITE))
                break
        while True:
            r: int = random.randint(0, 63)
            if (
                board.piece_at(r) is None
                and board.is_attacked_by(chess.WHITE, r) == False
            ):
                board.set_piece_at(r, chess.Piece(chess.KING, chess.BLACK))
                break
        output[f"Game {i}"] = {"startring_board": board.fen()}
        playing_loop(board, show_autoplay=show_games_played)
        output[f"Game {i}"]["result"] = board.result()
        output[f"Game {i}"]["moves"] = board.move_stack
        output[f"Game {i}"]["game_length"] = len(board.move_stack)
    return output


In [10]:
output = autplay(10000)
print(f"Results: {Counter([output[i]['result'] for i in output])}")
print(f"game length max: {max([output[i]['game_length'] for i in output])}")
print(f"game length median: {median([output[i]['game_length'] for i in output])}")
print(f"game length mean: {mean([output[i]['game_length'] for i in output])}")
with open("output.txt", "w") as f:
    f.write(json.dumps(str(output)))


Results: Counter({'1-0': 10})
game length max: 33
game length median: 19.0
game length mean: 19.6
